# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


# Data Collection and PreProcessing

In [2]:
# loading the data from the csv file to apandas dataframe
movies_info = pd.read_csv('/Users/saurabh453/Desktop/MRS/movies_data.csv')
movies_info

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [3]:
# printing the first 5 rows of the dataframe
movies_info.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
# number of rows and columns in the data frame

movies_info.shape

(4803, 24)

In [5]:
# selecting the relevant features for recommendation

columns_selected = ['genres','keywords','tagline','cast','director']
print(columns_selected)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [6]:
# replacing the null valuess with null string

for feature in columns_selected:
  movies_info[feature] = movies_info[feature].fillna('')

In [7]:
# combining all the 5 selected features

combined_columns = movies_info['genres']+' '+movies_info['keywords']+' '+movies_info['tagline']+' '+movies_info['cast']+' '+movies_info['director']

In [8]:
print(combined_columns)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [9]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [10]:
column_vectors = vectorizer.fit_transform(combined_columns)

In [11]:
print(column_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

# Cosine Similarity

In [12]:
# getting the similarity scores using cosine similarity

cosinesimilarity = cosine_similarity(column_vectors)

In [13]:
print(cosinesimilarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [14]:
print(cosinesimilarity.shape)

(4803, 4803)


# Getting movie name from user  

In [17]:
# getting the movie name from the user

movie_name = input('Favorite movie name : ')

Favorite movie name : terminator


In [18]:
# creating a list with all the movie names given in the dataset

titles_list = movies_info['title'].tolist()
print(titles_list)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [19]:
# finding the close match for the movie name given by the user

getting_close_match = difflib.get_close_matches(movie_name, titles_list)
print(getting_close_match)

['The Terminator', 'Terminator Genisys', 'Terminator Salvation']


In [20]:
close_match = getting_close_match[0]
print(close_match)

The Terminator


In [21]:
# finding the index of the movie with title

movie_index = movies_info[movies_info.title == close_match]['index'].values[0]
print(movie_index)

3439


In [22]:
# getting a list of similar movies

similarity_score = list(enumerate(cosinesimilarity[movie_index]))
print(similarity_score)

[(0, 0.12480340331169382), (1, 0.034584214898631685), (2, 0.0066396941531620055), (3, 0.03251013237509891), (4, 0.06836145403893923), (5, 0.02604835979288664), (6, 0.0), (7, 0.029328517578801845), (8, 0.011654119000802857), (9, 0.006399755354855168), (10, 0.1141397723702275), (11, 0.011143131400476743), (12, 0.017537195131619197), (13, 0.02318044886614199), (14, 0.09874753799596403), (15, 0.0), (16, 0.027809669026105526), (17, 0.00633874471994718), (18, 0.08221837559639687), (19, 0.014801088234824391), (20, 0.011729762911338384), (21, 0.005446368118487068), (22, 0.004776871009758756), (23, 0.00914695460313755), (24, 0.03239206920249841), (25, 0.05110596284613303), (26, 0.03026958073429826), (27, 0.03680180823675636), (28, 0.051091307787333035), (29, 0.03271766804095004), (30, 0.032045963060645576), (31, 0.03895056243302879), (32, 0.011046931581187993), (33, 0.03521911349832008), (34, 0.0), (35, 0.05373010069883703), (36, 0.04841510848909464), (37, 0.03784502713518707), (38, 0.006623118

In [24]:
len(similarity_score)

4803

In [25]:
# sorting the movies based on their similarity score

similar_movies_sorted = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(similar_movies_sorted)

[(3439, 1.0), (279, 0.38860434541925903), (108, 0.36342338341686814), (2403, 0.29843706562331657), (93, 0.2956594881073056), (43, 0.2325677038202857), (1725, 0.19418186280151034), (286, 0.18450601523001742), (2433, 0.18028960709200476), (634, 0.1789020092800192), (125, 0.1729432854412031), (2444, 0.1712313959877991), (3014, 0.16308572661113563), (4401, 0.16249149799075477), (322, 0.16101208066006162), (123, 0.15829193667829297), (510, 0.15750439394443255), (95, 0.15542093215931807), (282, 0.1542111450857223), (2047, 0.15383401578088585), (1959, 0.15167377243231853), (224, 0.14927762749024487), (422, 0.14884972659453488), (4266, 0.14874137241044352), (1608, 0.14586438323608739), (2654, 0.1457316934219378), (207, 0.14560278763155787), (1099, 0.14156612785583564), (4574, 0.1375594215945904), (1568, 0.1344689207999927), (1653, 0.13331737583999265), (196, 0.1317135925631783), (587, 0.12951392062059405), (1930, 0.12687822530352458), (1782, 0.12520660255054059), (0, 0.12480340331169382), (993

In [ ]:
# print the name of similar movies based on the index

print('Suggested Movies: \n')

i = 1

for movie in similar_movies_sorted:
  index = movie[0]
  title_index = movies_info[movies_info.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_index)
    i+=1

# Movie Recommendation System

In [ ]:

movie_name = input('Favorite movie name : ')

titles_list = movies_info['title'].tolist()

getting_close_match = difflib.get_close_matches(movie_name, titles_list)

close_match = getting_close_match[0]

movie_index = movies_info[movies_info.title == close_match]['index'].values[0]

similarity_score = list(enumerate(cosinesimilarity[movie_index]))

similar_movies_sorted = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Suggested Movies: \n')

i = 1

for movie in similar_movies_sorted:
  index = movie[0]
  title_index = movies_info[movies_info.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_index)
    i+=1